In [4]:
%pip install wikipedia

import wikipedia
wikipedia.set_lang("de")  # auf Deutsch umstellen

# Beispiel: Artikel laden
page = wikipedia.page("Mathematik")
print(page.title)
print(page.content[:1000])  # zeige ersten 1000 Zeichen


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\tobia\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.
Mathematik
Die Mathematik (bundesdeutsches Hochdeutsch: [matemaˈtiːk], [matemaˈtik]; österreichisches Hochdeutsch: [mateˈmaːtik]; von altgriechisch μαθηματικὴ τέχνη mathēmatikē téchnē ‚die Kunst des Lernens‘) ist eine Formalwissenschaft, die aus der Untersuchung von geometrischen Figuren und dem Rechnen mit Zahlen entstand. Für Mathematik gibt es keine allgemein anerkannte Definition; heute wird sie üblicherweise als eine Wissenschaft beschrieben, die durch logische Definitionen selbstgeschaffene abstrakte Strukturen mittels der Logik auf ihre Eigenschaften und Muster untersucht.


== Geschichte ==

Die Mathematik ist eine der ältesten Wissenschaften. Ihre erste Blüte erlebte sie noch vor der Antike in Mesopotamien, Indien und China, später in der Antike in Griechenland und im Hellenismus. Von dort datiert die Orientierung an der Aufgabenstellung des „rein logischen Beweisens“ und die erste Axiomatisierung, nämlich die e

In [1]:
import json

# JSON-Datei öffnen und laden
with open("Artikel.json", "r", encoding="utf-8") as file:
    data = json.load(file)

string_array = list(data)

print(string_array[0])

# Rechnen mit Quadratwurzeln Wurzeln kann man sowohl aus Zahlen als auch aus Termen ziehen. Aber auch beim Lösen von Gleichungen sind Wurzeln sehr wichtig. ## Quadratwurzel aus Termen Man kann Wurzeln nicht nur aus Zahlen, sondern auch aus Termen ziehen. Auch hier muss man beachten, dass der Radikand (= das, was unter der Wurzel steht) nicht negativ wird. Und genauso wie bei Quadratwurzeln von Zahlen ist die Quadratwurzel von Termen immer positiv oder 0. ### Beispiele (Hier stehen Beispiele als Formeln, nicht kopiert.) ### Definitionsmenge Beim Wurzelziehen aus Termen muss man darauf achten, dass der Radikand nicht negativ wird. Das heißt, man muss den Definitionsbereich beachten. ▸ Beispiele ### Wurzeln und der Betrag Steht unter der Wurzel ein Term, so muss man beim Radizieren den Betrag berücksichtigen, damit immer ein positiver Ausdruck herauskommt. #### Beispiele • Beachte Betragsstriche |x| wird zu $\lvert x \rvert$ Würde man den Betrag nicht setzen, also als Ergebnis schreiben $

In [8]:
import requests
import json
import re

OLLAMA_MODEL = "gpt-oss:20b"
OLLAMA_URL = "http://localhost:11434/api/generate"

def query_ollama(prompt, model=OLLAMA_MODEL):
    response = requests.post(OLLAMA_URL, json={
        "model": model,
        "prompt": prompt,
        "stream": False,
        "think": True
    })
    data = response.json()
    if "response" in data:
        return data["response"].strip()
    else:
        print("⚠️ Unerwartete Antwort:", data)
        return None

def split_into_chunks(text, max_chars=250):
    paragraphs = text.split("\n")
    chunks, current = [], ""
    for p in paragraphs:
        if len(current) + len(p) < max_chars:
            current += "\n" + p
        else:
            chunks.append(current.strip())
            current = p
    if current:
        chunks.append(current.strip())
    return chunks

def generate_qa_from_text(text_chunk):
    frage_prompt = f"""
Du bist ein Lernassistent für Schüler.  Lies den Text und gib dann eine passende Frage in deutscher Sprache aus. Wenn du keinen Text findest gibt eine andere Frage aus, die ein Schüler über Themen wie Wurzeln, Potenzen, Brüche, Gleichungen, Geometrie, Funktionen oder Statistik stellen könnte.

TEXT:
{text_chunk}

Gib nur die Frage aus, ohne weitere Erklärungen.
"""
    frage_result = query_ollama(frage_prompt).strip()
    frage = frage_result

    #if "</think>" in frage_result:
    #    frage = frage_result.split("...done thinking.", 1)[1].strip()
    #else:
    #    print("⚠️ Kein </think>-Block gefunden:\n", frage_result)
    #    return None, None

    antwort_prompt = f"""
Du bist ein Lernassistent für Schüler. Beantworte die folgende Frage auf Grundlage des bereitgestellten Textes. 
Gib ganz am Anfang einen <think> </think>Block mit deiner Überlegung aus, gefolgt von der finalen Antwort. Achte darauf, dass der think Block deiner Originalen Überlegung entspricht und nicht einfach nur eine Wiederholung der Frage ist. Zudem muss die Überlegung auf Englisch sein. Die finale Antwort soll in deutscher Sprache sein. Der <think> Block soll ganz am Anfang stehen, gefolgt von der Antwort.

TEXT:
{text_chunk}

FRAGE:
{frage}
"""
    raw_response = query_ollama(antwort_prompt)
    

    return frage, raw_response


def build_dataset_from_json(json_file,  output_file="output_dataset.jsonl"):
    with open(json_file, "r", encoding="utf-8") as f:
        articles = json.load(f)

    print(f"📄 Geladene Artikel: {len(articles)}")

    dataset = []
    for i , content in enumerate(articles[:2]):
        print(f"\n ({i+1}/{len(articles)}) Artikel verarbeiten...")
        chunks = split_into_chunks(content)
        for j, chunk in enumerate(chunks):
            print(f"   ➤ Generiere Q&A ({j+1}/{len(chunks)})...")
            frage, antwort = generate_qa_from_text(chunk)
            if frage and antwort:
                dataset.append({
                    "conversations": [
                        {"role": "user", "content": frage},
                        {"role": "assistant", "content": antwort}
                    ]
                })

    with open(output_file, "w", encoding="utf-8") as f:
        for item in dataset:
            f.write(json.dumps(item, ensure_ascii=False) + "\n")

    print(f"\n✅ Gesamt: {len(dataset)} Einträge gespeichert in {output_file}")


if __name__ == "__main__":
    build_dataset_from_json("Artikel.json")


📄 Geladene Artikel: 24

 (1/24) Artikel verarbeiten...
   ➤ Generiere Q&A (1/2)...
   ➤ Generiere Q&A (2/2)...

 (2/24) Artikel verarbeiten...
   ➤ Generiere Q&A (1/2)...
   ➤ Generiere Q&A (2/2)...

✅ Gesamt: 4 Einträge gespeichert in output_dataset.jsonl


In [3]:
import wikipedia

# Sprache auf Deutsch setzen
wikipedia.set_lang("de")

def zeige_wikipedia_artikel(fachbegriff, max_treffer=100):
    print(f"🔎 Suche Wikipedia-Artikel zu: {fachbegriff}")
    artikel = wikipedia.search(fachbegriff, results=max_treffer)
    
    if not artikel:
        print("❌ Keine Artikel gefunden.")
    else:
        print(f"✅ {len(artikel)} Artikel gefunden:")
        for i, titel in enumerate(artikel, 1):
            print(f"{i}. {titel}")

# Beispiel
zeige_wikipedia_artikel("Algebra, Gleichungen, Funktionen")

🔎 Suche Wikipedia-Artikel zu: Algebra, Gleichungen, Funktionen
✅ 100 Artikel gefunden:
1. Cardanische Formeln
2. Quadratische Gleichung
3. Satz von Vieta
4. Eigenwerte und Eigenvektoren
5. Algebraische Funktion
6. Binomische Formeln
7. Lineare Gleichung
8. Lösen von Gleichungen
9. Boolesche Algebra
10. Funktion (Mathematik)
11. Matrix (Mathematik)
12. Lineare Funktion
13. Lineare Algebra
14. Analysis
15. Algebraische Struktur
16. Laplace-Operator
17. Analytische Geometrie
18. Taschenrechner
19. Geradengleichung
20. Quadratische Ergänzung
21. Polynom
22. Polynom vierten Grades
23. Universelle Algebra
24. Kubische Gleichung
25. Gleichung
26. Algebra über einem Körper
27. Computeralgebrasystem
28. Körper (Algebra)
29. Komplexe Zahl
30. Komposition (Mathematik)
31. Funktionalanalysis
32. Zahlentheorie
33. Variable (Mathematik)
34. Lambertsche W-Funktion
35. Teilgebiete der Mathematik
36. Surjektive Funktion
37. Dirac-Gleichung
38. Ganzrationale Funktion
39. Poisson-Klammer
40. Al-Chwarizmi

Note: you may need to restart the kernel to use updated packages.


RepositoryNotFoundError: 401 Client Error. (Request ID: Root=1-687e82cf-080190a25815525863ed0e03;116c3664-8c18-4f29-9d23-677d78e18d1d)

Repository Not Found for url: https://huggingface.co/api/datasets/Tobronaut/Mathewiki/preupload/main.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated. For more details, see https://huggingface.co/docs/huggingface_hub/authentication
Invalid username or password.
Note: Creating a commit assumes that the repo already exists on the Huggingface Hub. Please use `create_repo` if it's not the case.